# 유전 알고리즘을 구현해 봅시다.
---

생물체가 환경에 적응하면서 진화해나가는 모습을 모방하여 최적의 해를 구하는 유전 알고리즘을 1강에서 배운 개념들을 활용해 *직접* 구현해 보겠습니다.  
GA는 딥러닝 모델 라기 보다는 수학적으로 정의 되지 않은 문제에 대해 최적화를 수행하는 방법론입니다.



GA를 이해하기 위해서는 사전에 정의하고 넘어가야 할 몇 가지 개념들이 존재합니다.
 
염색체(Chromosome) : 여러개의 유전자를 담고 있는 하나의 집합을 의미한다. GA에서는 하나의 해를 표현한다.  
  
유전자(Gene) : 염색체를 구성하고 있는 요소로서, 하나의 유전 정보를 나타낸다.  
예를 들어, 하나의 특정한 염색체가 [A,B,C,D] 라고 정의된다면 해당 염색체의 유전자는 A,B,C,D로 4개의 유전자가 존재한다.  

## 0과 3사이의 값을 가지는 6개의 유전자를 가진 염색체를 생성하는 함수 정의 하기

In [2]:
import random

# 0 ~ 3 사이의 랜덤한 값을 가지는 염색체를 생성하기.

def generate_chromosome(chromosome_num):
    pool = []
    for i in range(chromosome_num):
        chromosome = list( random.randint(0,3) for _ in range(6) )
        pool.append(chromosome)
    return pool

chromosomes = generate_chromosome(10) # 10 개 생성
chromosomes

[[2, 2, 2, 1, 3, 3],
 [1, 3, 3, 0, 1, 0],
 [0, 1, 0, 3, 3, 0],
 [3, 0, 3, 0, 2, 1],
 [3, 2, 0, 3, 2, 3],
 [0, 2, 0, 2, 3, 1],
 [3, 0, 3, 2, 3, 1],
 [0, 3, 3, 2, 0, 3],
 [3, 0, 3, 3, 0, 3],
 [3, 0, 3, 1, 0, 0]]

# 함수의 재사용성을 최대한 활용하려면?

함수는 반복되는 동작을 매번 작성하지 않고 필요할 때 호출해 코드의 복잡도를 낮추는데 매우 많은 도움이 됩니다.  
동일한 동작이지만 약간의 세부적인 조정등이 필요할 때에는 매개 변수를 통해 처리하면 좋습니다.   
  
예를 들어 언제는 6개의 유전자를 가진 10개의 유전체를 생성해야 할 때가 있고  
8개의 유전자를 가진 20개의 유전체를 생성해야 할 때가 있다고 할 때  
  
아래와 같이 gene_num 이라는 매개 변수를 추가하여 봅시다.


In [3]:
import random

def generate_chromosome(chromosome_num, gene_num):
    pool = []
    for i in range(chromosome_num):
        chromosome = list( random.randint(0,3) for _ in range(gene_num) )
        pool.append(chromosome)
    return pool

generate_chromosome(10, 6) # 6개의 유전체를 가진 염색체 10 개 생성

[[1, 2, 1, 0, 2, 2],
 [3, 1, 2, 2, 3, 2],
 [1, 1, 0, 2, 0, 2],
 [3, 2, 2, 2, 0, 2],
 [0, 0, 3, 1, 2, 2],
 [1, 0, 1, 0, 3, 1],
 [1, 1, 3, 0, 0, 1],
 [0, 1, 1, 2, 2, 0],
 [2, 0, 1, 2, 2, 0],
 [1, 3, 1, 1, 1, 1]]

# 인코딩이란 무엇일까?


사용자가 입력한 문자나 실생활에 사용되는 의미 기호들을 컴퓨터가 이용할 수 있는 신호로 만드는 것을 인코딩이라고 합니다.  
우리가 해결 하고자 하는 문제는 4 * 4의 블록을 가지는 지도에서 START 블록에서 출발해 END 블록에 도달하는 길을 찾는 것입니다.  
우리가 쉽게 이해하는 동서남북 방향이라는 개념을 컴퓨터는 이해하지 못합니다.  
따라서 방향에 따라 숫자값으로 인코딩을 해주어야 할 필요가 있습니다.  

인코딩에 따른 START에서 END에 도달하는 해(정답)은  
[ 0, 2, 0, 2, 2, 0 ] 입니다. (위 오른쪽 위 오른쪽 오른쪽 위)

# 특정 염색체를 해와 비교해 Score(점수)를 계산해 봅시다.


GA(유전 알고리즘)은 염색체와 유전체와 함께 적합도라는 중요한 개념이 존재합니다.  
이름과 같이 생성된 염색체가 얼마나 특정 환경(문제)에 적합 한지 판단하는 지표입니다.  
  
  
주어진 문제를 풀기위해 적합도를 판단하는 함수를 잘 선택하는 것이 모델의 성능을 크게 좌우합니다.  
먼저 정답을 보기전에 스스로 유전체의 적합도를 계산하는 함수를 작성해 봅시다.  
  


In [4]:
# 해답과 비교해 동일한 원소 개수 만큼을 score로 반환하는 함수.

def score(chromosome):
    
    equal_num = 0
    solution = [ 0, 2, 0, 2, 2, 0 ]
    
    for idx,x in enumerate(chromosome):
        if solution[idx] == x:
            equal_num += 1
            
    return equal_num

문제(START블락 에서 END블락에 도착하는 길을 찾는)를 분석한 결과  
해답([0,2,0,2,2,0])과 유전체별로 값을 비교해 동일한 유전체의 개수를 적합도로써 평가하기로 결정하였습니다.  
따라서 위와 같은 score 함수를 정의하고 특정 염색체([3,2,0,1,2,1])의 적합도를 산출해 봅시다.  

In [5]:
score([3,2,0,1,2,1])

3

# 함수의 재사용성 관점에서 score 함수를 조금 더 개선 해봅시다. (스스로)
---

해답이 항상 [0,2,0,2,2,0]의 값을 가지는 것은 아닙니다.    
따라서 해답을 매개변수로 받을 수도 있을 것입니다.  

In [6]:
def score(solution, chromosome):
    equal_num = 0
    
    for idx,x in enumerate(chromosome):
        if solution[idx] == x:
            equal_num += 1
            
    return equal_num

In [7]:
score(solution=[0,2,0,2,2,0],chromosome=[3,2,0,1,2,1])

3

# 하지만 매번 solution을 매개 변수로 전달하기에는 번거로움이 따릅니다. 
---

혹시 solution의 매개변수값을 한번만 전달하고 계속 재사용할 수는 없을까요?  
당연히 가능합니다. 

In [8]:
from functools import partial

score_func_0 = partial(score, [0,2,0,2,2,0])
score_func_0([3,2,0,1,2,1])

3

# 커링이란?
---

partial이라는 함수를 통해 score 함수의 첫번째 매개변수에 값을 bind 할 수 있습니다.  
이러한 기법은 함수지향 프로그래밍의 기본 컨셉중 하나입니다.  
함수를 자유자재로 가지고 노는 듯 하죠?  
이러한 개념을 커링이라고 합니다.  
매개변수를 미리 채워놓고 함수를 더 간단히 만들어 재사용하는 기법입니다.  

In [9]:
sorted(chromosomes, key=score_func_0)

[[3, 0, 3, 3, 0, 3],
 [2, 2, 2, 1, 3, 3],
 [1, 3, 3, 0, 1, 0],
 [3, 0, 3, 0, 2, 1],
 [3, 0, 3, 2, 3, 1],
 [3, 0, 3, 1, 0, 0],
 [0, 3, 3, 2, 0, 3],
 [0, 1, 0, 3, 3, 0],
 [3, 2, 0, 3, 2, 3],
 [0, 2, 0, 2, 3, 1]]

위와 같이 미리 만들어 놓은 10개의 염색체 변수를 가진 배열 chromosomes을  
적합도 기준으로 정렬해 봅시다.  
key로 전달된 score_func_0에 주목하여 봅시다.  
score_func_0는 해가 [0,2,0,2,2,0]인 경우의 적합도를 산출하는 함수입니다.  

In [10]:
score_func_1 = partial(score, [1,0,3,1,2,1])
sorted(chromosomes, key=score_func_1)

[[0, 1, 0, 3, 3, 0],
 [2, 2, 2, 1, 3, 3],
 [3, 2, 0, 3, 2, 3],
 [0, 2, 0, 2, 3, 1],
 [0, 3, 3, 2, 0, 3],
 [1, 3, 3, 0, 1, 0],
 [3, 0, 3, 3, 0, 3],
 [3, 0, 3, 2, 3, 1],
 [3, 0, 3, 1, 0, 0],
 [3, 0, 3, 0, 2, 1]]

만약 해가 [1,0,3,1,2,1]인 경우의 적합도를 구하는 함수는 위와 같이 재정의 할 수 있습니다.

# 유전 알고리즘은 어떻게 해(solution)을 찾아 나갈까?
---

자손 염색체를 생성할 때 흔히 적합도가 가장 높은 염색체들을 선택하는 것이 최고의 방법이라고 생각할 수 있습니다.  
하지만 이러한 방법은 염색체의 다양성을 손상시키기 때문에 전역 최적해(Global optimum)를 찾기에는 부적합합니다.  
이러한 문제를 피하기 위해서 GA에서는 룰렛 휠 선택(Roulette WheelSelection) 방법을 이용합니다.  
룰렛 휠 선택이란, 우리가 흔히 생각하는 원판을 돌리면서 확률에 기반해 결과가 도출되는 룰렛의 개념과 비슷하다고 생각하면 된다.  
밑의 그림은 룰렛 휠 선택에 대한 확률적 수식이며 다음 그림은 수식을 룰렛 그림으로 나타낸 그림이다.  
룰렛 그림에서 면적의 총 합은 1(100%)이다.  왜냐하면 각각이 확률값이며 확률의 합은 1이기 때문이다.


![rullet](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FwGvLN%2FbtqFtw0tkn9%2FsJGMgRA34Bvqz3PfTsxeKK%2Fimg.png)

위와 같이 적합도에 비례하는 확률값을 기반으로 임의의 염색체를 선택하는 함수를 작성해 봅시다.  
함수의 시그니쳐는 아래와 같습니다.  
염색체 집합을 첫번째 매개변수로 입력받고 적합도 함수를 두번째,  
선택하고자 하는 염색체 개수를 세번째 매개변수로 받습니다.  


def pick(chromosomes, score_func, pick_num)  

강의 분량상 중복은 허용하도록 하겠습니다.

# random.randint 함수를 응용하여 확률 기반 선택 알고리즘을 작성해 봅시다. 
---


In [11]:
def make_rullet(chromosomes, score_func):
    scores = list(score_func(x) for x in chromosomes)
    print(f"scores: {scores}")
    chromosome_num = len(chromosomes)
    rullets = list( 0. for _ in range(chromosome_num))
    
    for idx, score in enumerate(scores):
        
        for ridx in range(idx, chromosome_num):
            rullets[ridx] += score
                
    return rullets

make_rullet(chromosomes, score_func_0)

scores: [1, 1, 3, 1, 3, 4, 1, 2, 0, 1]


[1.0, 2.0, 5.0, 6.0, 9.0, 13.0, 14.0, 16.0, 16.0, 17.0]

어떻게 하면 적합도와 비례하는 확률 기반으로 임의의 염색체를 선택하는 함수를 작성할 수 있을까요?  
물론 항상 프로그래밍에 정답은 없습니다.  
예시의 코드는 문제를 해결하는 하나의 방법일 뿐이라는 점을 꼭 염두해 주세요.  
  
저는 make_rullet이라는 이름의 함수를 먼저 생성하였습니다.
그리고 나서 각각의 염색체 적합도를 더해 나가는 (np.cumsum과 동일) 방식으로 룰렛을 먼저 생성하였습니다.

첫번째 염색체의 적합도가 1이라면 먼저 rullet을 아래와 같이 업데이트 합니다.

[ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ]

그리고 나서 두번째의 염색체의 적합도가 2라면 rullet을 다시 아래와 같이 업데이트 합니다.

[ 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, ]

세번째의 적합도가 3이라면,

[ 1, 3, 6, 6, 6, 6, 6, 6, 6, 6, ] 으로 업데이트 합니다.

이러한 과정을 모든 염색체에 대해 반복한 후 아래와 같은 룰렛을 만들었습니다. 

[1.0, 3.0, 6.0, 7.0, 8.0, 8.0, 10.0, 12.0, 13.0, 16.0]

In [16]:
from random import randint

def pick(chromosomes, score_func, pick_num):
    rullet = make_rullet(chromosomes, score_func)
    rullet_max = max(rullet) # 또는 rullet[-1]
    picked = []
    
    for _ in range(pick_num):
        seed = randint(0, rullet_max)
        for idx, rullet_value in enumerate(rullet):
            if idx == 0 and seed <= rullet_value:
                picked.append(idx)
                            
            if seed <= rullet_value and seed > rullet[idx-1]:
                picked.append(idx)
                break
    
    return picked
    
pick(chromosomes, score_func_0, 3)

scores: [1, 1, 3, 1, 3, 4, 1, 2, 0, 1]


[9, 3, 7]

그리고 임의의 염색체를 뽑을때마다 0부터 rullet의 마지막 값을 범위로 
임의의 정수값을 randint 함수를 통해 구합니다.  (seed)  
그리고 나서 rullet을 순회하면서 임의의 정수값이 포함되는 구간을 구합니다.  
해당 구간의 idx가 뽑힌 염색체의 chromosomes 배열의 인덱스인 것입니다.  
  
조금 어려울 수도 있을것 같지만, 실제로 작동하는 GA 모델을 완성하고난 성취감을 위해 조금만 더 달려봅시다.

# 염색체를 변형하여 자손을 생성해 봅시다.
---

방금 언급했던 룰렛 휠 선택 방법으로 선정된 두 개의 부모 염색체들로부터 하나의 자손 염색체를 생성합니다.  
이 때 GA에서는 자손 염색체를 생성하는 연산으로서 주로 Crossover이라는 연산을 사용합니다.  
Crossover 연산에 대해 그림으로 설명하자면 다음과 같습니다.  

![crossover](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FuoxLZ%2FbtqFvvsl8UK%2FHLxWs91QXgg3CaK66swduk%2Fimg.png)

선택된 두개의 염색체를 임의의 점 기준으로 서로 이어 붙이는 것이죠.  

In [17]:
def crossover(ch0, ch1):
    division_point = randint(0, len(ch0) - 1)
    return ch0[:division_point] + ch1[division_point:]

crossover([1,2,3,4,5,6],[6,5,4,3,2,1])

[1, 2, 3, 4, 5, 1]

# 이제 GA알고리즘을 완성할 거의 모든것을 작성하였습니다.
---

유전알고리즘을 구현하기 위해 필요한 요소들을 직접 작성해 보았습니다.  
작성한 함수는 아래와 같습니다.  

- 임의의 유전체 생성
- 적합도 판별
- 적합도 기반 임의 유전체 선택
- 유전체 변형

속성(Attribute) 기준으로 봤을때 아래와 같은 요소들이 있습니다.

- 염색체
- 유전자
- 해(solution) 염색체


# Class가 필요해요.
---

class란 여러 기능과 속성 집합을 가진 하나의 객체라고 알아본 바 있습니다.  
유전알고리즘 자체를 하나의 class로 표현한다면 생산성 높은 코드를 작성할 수 있을 것입니다.

먼저 Class를 정의할 때에는 기능과 속성을 분리하고  
속성은 또 다시 공통 속성과 일반 속성으로 분리하여 생각하면 쉽습니다.  

In [30]:
def score(solution, chromosome):
    equal_num = 0
    
    for idx,x in enumerate(chromosome):
        if solution[idx] == x:
            equal_num += 1
            
        return equal_num


class GA:
    
    model_name = "Genetic Algorithm"
    
    def __init__(self, chromosome_num, gene_num, solution):
        self.chromosomes = generate_chromosome(chromosome_num, gene_num)
        self.solution = solution
    
        self.score_func = partial(score, solution) # 해당 클래스는 score 함수의 첫번째 인자를 해(solution)로 커링한다.
    
    def generate_chromosomes(self, chromosome_num, gene_num):
        pool = []
        
        for i in range(chromosome_num):
            chromosome = list( random.randint(0,3) for _ in range(gene_num) )
            pool.append(chromosome)
        
        return pool
    

    def make_rullet(self, chromosomes):
        scores = list(self.score_func(x) for x in chromosomes) # map 으로도 구현이 가능하다.
        
        chromosome_num = len(chromosomes)
        rullets = list( 0. for _ in  range(chromosome_num))
    
        for idx, score in enumerate(scores):
        
            for ridx in range(idx, chromosome_num):
                rullets[ridx] += score
                
        return rullets

    def pick(self, pick_num):
        rullet = self.make_rullet(self.chromosomes)
        rullet_max = max(rullet) # 또는 rullet[-1]
        picked = []

        for _ in range(pick_num):
            seed = randint(0, rullet_max)
            for idx, rullet_value in enumerate(rullet):
                if idx == 0 and seed <= rullet_value:
                    picked.append(idx)

                if seed <= rullet_value and seed > rullet[idx-1]:
                    picked.append(idx)
                    break

        return picked

### 위와 같이 속성(Attribute)와 기능(method)를 포함하는 하나의 GA 클래스를 생성해보았습니다.
---

아래 코드와 같이 클래스를 생성하고 염색체를 생성자에서 초기화된 염색체를 확인해 봅시다. 

In [31]:
ga_model = GA(10, 6, solution=[0,2,0,2,2,0])
ga_model.chromosomes

[[3, 1, 1, 0, 0, 3],
 [1, 0, 3, 2, 0, 2],
 [2, 0, 0, 3, 0, 3],
 [0, 3, 0, 1, 0, 3],
 [0, 3, 0, 3, 2, 0],
 [0, 0, 3, 0, 0, 0],
 [2, 2, 1, 0, 2, 0],
 [1, 1, 3, 1, 3, 1],
 [1, 0, 0, 3, 2, 1],
 [2, 3, 3, 1, 2, 0]]

위 코드는 6개의 유전체를 가진 10개의 염색체를 가지고 solution을 찾아나가는 GA 인스턴스를 생성하는 코드입니다.  
그렇다면 이제 적합도 기반의 확률로 3개의 염색체를 뽑아 보도록 하겠습니다.  

In [57]:
ch_idx_0, ch_idx_1 = ga_model.pick(2) # 2개의 원소를 가지는 배열을 이렇게 unpacking 할 수 있다.

In [58]:
ch_idx_0

3

In [59]:
ch_idx_1

8

아래와 같이 GA 클래스에 crossover static 함수를 추가합니다.  
static으로 선언한 이유는 crossover가 내부 상태(속성)을 참조하지 않고 연산할 2개의 염색체를 인자로 받기 때문입니다.  

    @staticmethod
    def crossover(cls, ch0, ch1):
        division_point = randint(0, len(ch0) - 1)
        return ch0[:division_point] + ch1[division_point:]

In [46]:
def score(solution, chromosome):
    equal_num = 0
    
    for idx,x in enumerate(chromosome):
        if solution[idx] == x:
            equal_num += 1
            
        return equal_num


class GA:
    
    model_name = "Genetic Algorithm"
    
    def __init__(self, chromosome_num, gene_num, solution):
        self.chromosomes = generate_chromosome(chromosome_num, gene_num)
        self.solution = solution
    
        self.score_func = partial(score, solution) # 해당 클래스는 score 함수의 첫번째 인자를 해(solution)로 커링한다.
    
    def generate_chromosomes(self, chromosome_num, gene_num):
        pool = []
        
        for i in range(chromosome_num):
            chromosome = list( random.randint(0,3) for _ in range(gene_num) )
            pool.append(chromosome)
        
        return pool
    

    def make_rullet(self, chromosomes):
        scores = list(self.score_func(x) for x in chromosomes) # map 으로도 구현이 가능하다.
        
        chromosome_num = len(chromosomes)
        rullets = list( 0. for _ in  range(chromosome_num))
    
        for idx, score in enumerate(scores):
        
            for ridx in range(idx, chromosome_num):
                rullets[ridx] += score
                
        return rullets

    def pick(self, pick_num):
        rullet = self.make_rullet(self.chromosomes)
        rullet_max = max(rullet) # 또는 rullet[-1]
        picked = []

        for _ in range(pick_num):
            seed = randint(0, rullet_max)
            for idx, rullet_value in enumerate(rullet):
                if idx == 0 and seed <= rullet_value:
                    picked.append(idx)

                if seed <= rullet_value and seed > rullet[idx-1]:
                    picked.append(idx)
                    break

        return picked
    
    @staticmethod
    def crossover(ch0, ch1):
        division_point = randint(0, len(ch0) - 1)
        return ch0[:division_point] + ch1[division_point:]

In [60]:
ga_model = GA(10, 6, solution=[0,2,0,2,2,0])
ga_model.chromosomes

ch_idx_0, ch_idx_1 = ga_model.pick(2) 

ch0 = ga_model.chromosomes[ch_idx_0]
ch1 = ga_model.chromosomes[ch_idx_1]


GA.crossover(ch0, ch1)

[1, 0, 2, 2, 0, 3]

이러한 연산을 반복하여 정답인 [ 0, 2, 0, 2, 2, 0 ] 에 가까워 질때 까지 이러한 연산을 반복 합니다.  